In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OrdinalEncoder
import re

In [2]:
df1 = pd.read_csv("job_descriptions.csv")[:1000].drop(columns=["Job Id", 
                                                             "Experience", 
                                                             "Qualifications", 
                                                             "latitude", 
                                                             "longitude", 
                                                             "Company Size",
                                                             "Benefits", 
                                                             "Company Profile",
                                                             "Job Posting Date",
                                                             "Preference",
                                                             "skills",
                                                              "Job Description"])
print(df1.shape)

(1000, 11)


In [3]:
df2 = pd.read_excel("Education_Experience.xlsx").drop(columns=["Requirements"])
print(df2.shape)

(178, 4)


In [4]:
combined_df = pd.concat([df2, df1], axis=1)

In [5]:
combined_df

,EDUCATION,EXPERIENCE,SKILLS,Job Description,Salary Range,location,Country,Work Type,Contact Person,Contact,Job Title,Role,Job Portal,Responsibilities,Company
0,A relevant Bachelor's degree,A minimum of 3 years of relevant experience,"Proficiency in Python for data analysis, machi...","We’re a diverse team of more than 3,000 tech e...",$59K-$99K,Douglas,Isle of Man,Intern,Brandon Cunningham,001-381-930-7517x737,Digital Marketing Specialist,Social Media Manager,Snagajob,"Manage and grow social media accounts, create ...",Icahn Enterprises
1,Bachelor's or Master degree,2-6 years of relevant work experience in data ...,The insight to take ambiguous problems and sol...,"Working across industries and disciplines, we ...",$56K-$116K,Ashgabat,Turkmenistan,Intern,Francisco Larsen,461-509-4216,Web Developer,Frontend Web Developer,Idealist,"Design and code user interfaces for websites, ...",PNC Financial Services Group
2,"Master’s Degree or higher in Computer Science,...","2+ years of experience in data analytics, cons...",Proficiency in at least one programming langua...,"We go beyond what was once thought possible, c...",$61K-$104K,Macao,"Macao SAR, China",Temporary,Gary Gibson,9687619505,Operations Manager,Quality Control Manager,Jobs2Careers,Establish and enforce quality control standard...,United Services Automobile Assn.
3,Bachelor’s or Master’s degree in Computer Scie...,At least 2 years of deep-learning/NLP experien...,"Very experienced with SQL queries, Basic ETL, ...",Leveraging BCG’s global network and partnershi...,$65K-$91K,Porto-Novo,Benin,Full-Time,Joy Lucero,+1-820-643-5431x47576,Network Engineer,Wireless Network Engineer,FlexJobs,"Design, configure, and optimize wireless netwo...",Hess
4,"Tertiary qualifications in ICT, Computer Scien...",Minimum of 3 years of industry experience in D...,Proficiency in one or more analytics & visuali...,"Together, we strive to create solutions that w...",$64K-$87K,Santiago,Chile,Intern,Julie Johnson,343.975.4702x9340,Event Manager,Conference Manager,Jobs2Careers,Specialize in conference and convention planni...,Cairn Energy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,NaN,NaN,NaN,NaN,$59K-$117K,Amsterdam,Netherlands,Full-Time,Tracy Williams,381-936-1537x100,Urban Planner,Environmental Planner,Stack Overflow Jobs,Focus on environmental impact assessments and ...,BHEL (Bharat Heavy Electricals Limited)
996,NaN,NaN,NaN,NaN,$60K-$104K,Kyiv (also known as Kiev),Ukraine,Part-Time,Anthony George,+1-743-932-9115x356,Mechanical Engineer,Mechanical Design Engineer,FlexJobs,"Design mechanical systems, components, and pro...",CHS
997,NaN,NaN,NaN,NaN,$58K-$113K,Bandar Seri Begawan,Brunei,Temporary,Nathan Preston,857.609.5185x30668,IT Manager,IT Director,LinkedIn,Provide strategic leadership for IT department...,United Breweries (UB)
998,NaN,NaN,NaN,NaN,$61K-$128K,Kuala Lumpur,Malaysia,Full-Time,Samantha Miller,001-779-596-2760x47948,UI Developer,Front-End Developer,Jobs2Careers,Develop and maintain user interfaces for websi...,Brambles Limited


In [6]:
combined_df.head()
combined_df.columns =["EDUCATION", "EXPERIENCE", "SKILLS", "JOB DESCRIPTION", "SALARY RANGE", "LOCATION", "COUNTRY", "WORK TYPE", "CONTACT PERSON", "CONTACT", "JOB TITLE", "ROLE", "JOB PORTAL", "RESPONSIBILITIES", "COMPANY"]

In [7]:
combined_df = combined_df[["EDUCATION", "EXPERIENCE", "SKILLS", "RESPONSIBILITIES", "JOB DESCRIPTION"]]

In [8]:
combined_df

,EDUCATION,EXPERIENCE,SKILLS,RESPONSIBILITIES,JOB DESCRIPTION
0,A relevant Bachelor's degree,A minimum of 3 years of relevant experience,"Proficiency in Python for data analysis, machi...","Manage and grow social media accounts, create ...","We’re a diverse team of more than 3,000 tech e..."
1,Bachelor's or Master degree,2-6 years of relevant work experience in data ...,The insight to take ambiguous problems and sol...,"Design and code user interfaces for websites, ...","Working across industries and disciplines, we ..."
2,"Master’s Degree or higher in Computer Science,...","2+ years of experience in data analytics, cons...",Proficiency in at least one programming langua...,Establish and enforce quality control standard...,"We go beyond what was once thought possible, c..."
3,Bachelor’s or Master’s degree in Computer Scie...,At least 2 years of deep-learning/NLP experien...,"Very experienced with SQL queries, Basic ETL, ...","Design, configure, and optimize wireless netwo...",Leveraging BCG’s global network and partnershi...
4,"Tertiary qualifications in ICT, Computer Scien...",Minimum of 3 years of industry experience in D...,Proficiency in one or more analytics & visuali...,Specialize in conference and convention planni...,"Together, we strive to create solutions that w..."
...,...,...,...,...,...
995,NaN,NaN,NaN,Focus on environmental impact assessments and ...,NaN
996,NaN,NaN,NaN,"Design mechanical systems, components, and pro...",NaN
997,NaN,NaN,NaN,Provide strategic leadership for IT department...,NaN
998,NaN,NaN,NaN,Develop and maintain user interfaces for websi...,NaN


In [9]:
##---Preprocess Into Long Format---##

long_df = combined_df.melt(var_name="Category", value_name="Text")

long_df = long_df.dropna().reset_index(drop=True)

long_df

,Category,Text
0,EDUCATION,A relevant Bachelor's degree
1,EDUCATION,Bachelor's or Master degree
2,EDUCATION,"Master’s Degree or higher in Computer Science,..."
3,EDUCATION,Bachelor’s or Master’s degree in Computer Scie...
4,EDUCATION,"Tertiary qualifications in ICT, Computer Scien..."
...,...,...
1396,JOB DESCRIPTION,"As such, we welcome applications from people w..."
1397,JOB DESCRIPTION,"If you have the skills, curiosity and an adap..."
1398,JOB DESCRIPTION,Should you require any specific support or adj...
1399,JOB DESCRIPTION,"For this role, only those candidates with the ..."


### Vectorize

In [11]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(long_df["Text"])
vocabulary = vectorizer.vocabulary_
vectorizer.get_feature_names_out()

array(['000', '00am', '10', ..., 'zoning', 'zoology', 'zoom'],
      dtype=object)

In [12]:
tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

In [13]:
long_df = long_df.drop(columns=["Text"])

train_data = pd.concat([tfidf_df, long_df], axis=1)

train_data

,000,00am,10,100,12,12d,14,15,1year,20,...,you,young,your,zealand,zero,zone,zoning,zoology,zoom,Category
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,EDUCATION
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,EDUCATION
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,EDUCATION
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,EDUCATION
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,EDUCATION
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,JOB DESCRIPTION
1397,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.144505,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,JOB DESCRIPTION
1398,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.174386,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,JOB DESCRIPTION
1399,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,JOB DESCRIPTION


In [14]:
vocabulary

{'relevant': 1835,
 'bachelor': 235,
 'degree': 617,
 'or': 1514,
 'master': 1349,
 'higher': 1053,
 'in': 1112,
 'computer': 467,
 'science': 1942,
 'data': 595,
 'related': 1828,
 'field': 895,
 'statistics': 2101,
 'mathematics': 1354,
 'artificial': 182,
 'intelligence': 1172,
 'machine': 1320,
 'learning': 1270,
 'tertiary': 2206,
 'qualifications': 1759,
 'ict': 1084,
 'information': 1142,
 'systems': 2174,
 'retrieval': 1886,
 'nlp': 1464,
 'vision': 2373,
 'multimodal': 1435,
 'fields': 896,
 'ai': 104,
 'engineering': 783,
 'formal': 938,
 'education': 739,
 'technical': 2195,
 'physics': 1605,
 'etc': 819,
 'with': 2430,
 'an': 128,
 'emphasis': 762,
 'on': 1491,
 'software': 2041,
 'development': 654,
 'of': 1482,
 'business': 314,
 'administration': 67,
 'masters': 1350,
 'arts': 185,
 'applications': 159,
 'technology': 2199,
 'similar': 2023,
 'commerce': 433,
 'doctor': 699,
 'philosophy': 1601,
 'bs': 305,
 'study': 2132,
 'equivalent': 809,
 'practical': 1651,
 'experi

In [15]:
encoder = OrdinalEncoder()

encoder.fit(train_data[["Category"]])

OrdinalEncoder()

In [16]:
train_data['Category'] = encoder.transform(train_data[['Category']])

In [17]:
train_data

,000,00am,10,100,12,12d,14,15,1year,20,...,you,young,your,zealand,zero,zone,zoning,zoology,zoom,Category
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1397,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.144505,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1398,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.174386,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1399,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [18]:
train_data["Category"].value_counts()

Category
3.0    1000
2.0     178
4.0      86
1.0      76
0.0      61
Name: count, dtype: int64

### Train

In [20]:
from sklearn.ensemble import RandomForestClassifier

In [21]:
X_train = train_data.iloc[:, :-1]
Y_train = train_data.iloc[:, -1]

In [22]:
rf = RandomForestClassifier()

In [23]:
rf.fit(X_train, Y_train)

RandomForestClassifier()

### Testing

In [25]:
texts = [("At least 2 years of experience", 5),
        ("Honours degree in computer science", 4)]

In [26]:
tfidf_transformed_documents = vectorizer.transform([text[0] for text in texts])

In [27]:
test_data = pd.DataFrame(tfidf_transformed_documents.toarray(), columns=vectorizer.get_feature_names_out())

In [28]:
test_categories = pd.DataFrame([category[1] for category in texts], columns=["Category"])

In [29]:
test_data = pd.concat([test_data, test_categories], axis=1)

test_data = test_data.reset_index(drop=True)

In [30]:
X_test = test_data.iloc[:, :-1]
Y_test = test_data.iloc[:, -1]

In [31]:
X_test

,000,00am,10,100,12,12d,14,15,1year,20,...,years,you,young,your,zealand,zero,zone,zoning,zoology,zoom
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.448441,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
y_pred = rf.predict(X_test)

In [33]:
y_pred

array([1., 0.])

In [34]:
from sklearn.metrics import confusion_matrix

In [35]:
confusion_matrix(Y_test, y_pred)

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0]], dtype=int64)

### Chunk and Classify Job Listing

In [103]:
job_listing = """
Melbourne, Sydney

Data Scientist - Junior, Mid and Senior Opportunities
Choosing Capgemini means choosing a company where you will be empowered to shape your career in the way you’d like, where you’ll be supported and inspired by a collaborative community of colleagues around the world, and where you’ll be able to reimagine what’s possible. Join us and help the world’s leading organizations unlock the value of technology and build a more sustainable, more inclusive world.

Job Description
About Capgemini:

Capgemini is a diverse collective of more than 350,000 strategic and technological experts based across more than 50 countries, partnering with world-renowned clients to transform and manage their businesses.

We are dedicated to leveraging cloud, data, AI, connectivity, software, digital engineering, and platforms to address the entire breadth of their business needs. This passion drives a powerful commitment - to unlock the true value of technology.

Our business in Australia and New Zealand has over 3,000 team members devoted to helping clients get the future they want.

Now is the time to join our rapidly growing team who are at the forefront of finding new ways technology can help us reimagine what’s possible, collecting unique career experiences with global brands and game-changing tech projects.

About the Service Line:

Capgemini Global Insights & Data business line is a market leader in Data Engineering, Cloud Data Platforms, Data Science, and AI and Advanced Analytics across all sectors including financial services, public sector, consumer products, telecommunication & energy resources. Our offerings include end-2-end data integration to cloud platforms, complete suite of data engineering capabilities and a category of sector-based advanced analytics and AI driven solutions.

This is a unique opportunity within the Insights & Data (I&D) Global Business Line (GBL) to help grow the data, AI and Data Science engagements across all sectors in Australia and New Zealand. You will help shape and deliver complex projects and programs of work for our clients. You will understand our client’s need to bring maximum value, impact, and innovation to the market as you oversee the delivery of client engagements for data and AI initiatives.

Let’s talk about the roles and responsibilities:


Experience working in the field of generative AI applying large language models to real world problems.
Experience working with agentic AI frameworks such as Langgraph, Autogen and CrewAI etc.
Build data science and AI solutions for clients relating to their business problems in areas such as generative AI, NLP, computer vision, machine learning, etc.
Collaborate with sales and engagement teams to build client proposals in response to RFPs.
Experience in designing end-to-end Generative AI/ Data Science solutions.
Experience and Qualifications:
We’re interested in hearing from people across junior, mid and senior levels.
For Mid and senior levels, we are seeking extensive experience in AI and Machine Learning: Proven track record of leading, developing and implementing AI and machine learning solutions across various industries.
Strong Programming Skills: Proficiency in Python programming
Experience in Data Analysis and Visualization: Strong skills in data analysis, visualization, and tools such as Tableau, Power BI, and other data visualization platforms(preferred).
Degree in Data Science or Related Field: Bachelor’s or master’s in data science, Computer Science, Statistics, or a related field.
Due to the nature of the role, Australian citizens are highly desirable.
What You’ll Love About Working Here:
Exposure to transformational programs in AI & Data portfolio.
Career growth through learning platforms, certifications, and global mobility.
Inclusive culture backed by our “Inclusive Future for All” commitment.
Competitive total rewards and recognition programs.
Capgemini is an AI-powered global business and technology transformation partner, delivering tangible business value. We imagine the future of organizations and make it real with AI, technology and people. With our strong heritage of nearly 60 years, we are a responsible and diverse group of 420,000 team members in more than 50 countries. We deliver end-to-end services and solutions with our deep industry expertise and strong partner ecosystem, leveraging our capabilities across strategy, technology, design, engineering and business operations. The Group reported 2024 global revenues of €22.1 billion.
Make it real | www.capgemini.com

Ref. code
403925-en_GB
Posted on
29 Jan 2026
Experience level
Experienced Professionals
Contract type
Permanent
Location
Melbourne, Sydney
Business unit
I and D Global Business Line
Brand
Capgemini
Professional communities
Data & AI

"""

In [105]:
def chunk_text(text: str) -> list[str]:
    regex = re.compile(r"[\n:.]")
    chunks = re.split(regex, text)
    return chunks

In [107]:
def clean_chunks(chunks: list[str]) -> list[str]:
    cleaned_chunks = []
    for chunk in chunks:
        chunk = chunk.strip()
        if chunk:
            cleaned_chunks.append(chunk)
        
    return cleaned_chunks

In [109]:
def vectorize_chunks(chunks: list[str], vectorizer: TfidfVectorizer()) -> pd.DataFrame:
    vectorized_chunks = vectorizer.transform([chunk for chunk in chunks])
    vectorized_chunks_df = pd.DataFrame(vectorized_chunks.toarray(), columns=vectorizer.get_feature_names_out())
    return vectorized_chunks_df

In [111]:
def classify_chunks(vectorized_chunks: pd.DataFrame, rf: RandomForestClassifier()) -> list[int]:
    results = rf.predict(vectorized_chunks)
    return results

In [113]:
chunks = chunk_text(text=job_listing)

In [115]:
chunks = clean_chunks(chunks=chunks)

In [117]:
job_listing_vectorized_chunks_df = vectorize_chunks(chunks=chunks, vectorizer=vectorizer)

In [119]:
results = classify_chunks(vectorized_chunks=job_listing_vectorized_chunks_df, rf=rf)

In [121]:
decoded = encoder.inverse_transform(results.reshape(-1, 1))
print("\nDecoded Data:")
print(decoded)


Decoded Data:
[['JOB DESCRIPTION']
 ['RESPONSIBILITIES']
 ['JOB DESCRIPTION']
 ['JOB DESCRIPTION']
 ['JOB DESCRIPTION']
 ['JOB DESCRIPTION']
 ['JOB DESCRIPTION']
 ['JOB DESCRIPTION']
 ['JOB DESCRIPTION']
 ['JOB DESCRIPTION']
 ['JOB DESCRIPTION']
 ['JOB DESCRIPTION']
 ['JOB DESCRIPTION']
 ['JOB DESCRIPTION']
 ['JOB DESCRIPTION']
 ['JOB DESCRIPTION']
 ['JOB DESCRIPTION']
 ['JOB DESCRIPTION']
 ['EXPERIENCE']
 ['EXPERIENCE']
 ['JOB DESCRIPTION']
 ['RESPONSIBILITIES']
 ['EXPERIENCE']
 ['EXPERIENCE']
 ['EDUCATION']
 ['EXPERIENCE']
 ['RESPONSIBILITIES']
 ['SKILLS']
 ['SKILLS']
 ['EXPERIENCE']
 ['SKILLS']
 ['EDUCATION']
 ['EDUCATION']
 ['JOB DESCRIPTION']
 ['JOB DESCRIPTION']
 ['JOB DESCRIPTION']
 ['JOB DESCRIPTION']
 ['JOB DESCRIPTION']
 ['RESPONSIBILITIES']
 ['JOB DESCRIPTION']
 ['JOB DESCRIPTION']
 ['JOB DESCRIPTION']
 ['JOB DESCRIPTION']
 ['JOB DESCRIPTION']
 ['JOB DESCRIPTION']
 ['JOB DESCRIPTION']
 ['JOB DESCRIPTION']
 ['JOB DESCRIPTION']
 ['JOB DESCRIPTION']
 ['JOB DESCRIPTION']
 ['JOB

In [123]:
chunks

['Melbourne, Sydney',
 'Data Scientist - Junior, Mid and Senior Opportunities',
 'Choosing Capgemini means choosing a company where you will be empowered to shape your career in the way you’d like, where you’ll be supported and inspired by a collaborative community of colleagues around the world, and where you’ll be able to reimagine what’s possible',
 'Join us and help the world’s leading organizations unlock the value of technology and build a more sustainable, more inclusive world',
 'Job Description',
 'About Capgemini',
 'Capgemini is a diverse collective of more than 350,000 strategic and technological experts based across more than 50 countries, partnering with world-renowned clients to transform and manage their businesses',
 'We are dedicated to leveraging cloud, data, AI, connectivity, software, digital engineering, and platforms to address the entire breadth of their business needs',
 'This passion drives a powerful commitment - to unlock the true value of technology',
 'Our

In [125]:
x = pd.DataFrame(chunks)
y = pd.DataFrame(decoded)

In [127]:
results_df = pd.concat([x, y], axis=1)
results_df.columns = ["Text", "Category"]

In [129]:
results_df

,Text,Category
0,"Melbourne, Sydney",JOB DESCRIPTION
1,"Data Scientist - Junior, Mid and Senior Opport...",RESPONSIBILITIES
2,Choosing Capgemini means choosing a company wh...,JOB DESCRIPTION
3,Join us and help the world’s leading organizat...,JOB DESCRIPTION
4,Job Description,JOB DESCRIPTION
...,...,...
60,I and D Global Business Line,RESPONSIBILITIES
61,Brand,JOB DESCRIPTION
62,Capgemini,JOB DESCRIPTION
63,Professional communities,JOB DESCRIPTION


In [131]:
import docx
from docx.enum.text import WD_COLOR_INDEX

In [133]:
doc = docx.Document()

doc.add_heading('Job Listing', 0)

for _, text in results_df.iterrows():
    if text[1] == "SKILLS":
       doc.add_paragraph().add_run(text[0]).font.highlight_color = WD_COLOR_INDEX.YELLOW
    elif text[1] == "EDUCATION":
        doc.add_paragraph().add_run(text[0]).font.highlight_color = WD_COLOR_INDEX.RED
    elif text[1] == "EXPERIENCE":
        doc.add_paragraph().add_run(text[0]).font.highlight_color = WD_COLOR_INDEX.BLUE
    elif text[1] == "RESPONSIBILITIES":
        doc.add_paragraph().add_run(text[0]).font.highlight_color = WD_COLOR_INDEX.GREEN
    else:
        doc.add_paragraph(text[0])

doc.save('job_listing.docx')

C:\Users\yanni\AppData\Local\Temp\ipykernel_29464\1563920444.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if text[1] == "SKILLS":
C:\Users\yanni\AppData\Local\Temp\ipykernel_29464\1563920444.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif text[1] == "EDUCATION":
C:\Users\yanni\AppData\Local\Temp\ipykernel_29464\1563920444.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif text[1] == "EXPERIENCE":
C:\Users\yann